In [1]:
import itertools

from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [2]:
cbt_dataset = load_dataset('cbt','CN')

Found cached dataset cbt (C:/Users/34941/.cache/huggingface/datasets/cbt/CN/1.1.0/dc4451a8a4b50cebb78fdb19fa9f964b27fcdcef915467b8b7055a3a8d8cef7b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
train_set = cbt_dataset['train']
val_set = cbt_dataset['validation']
test_set = cbt_dataset['test']

In [4]:
train_set

Dataset({
    features: ['sentences', 'question', 'answer', 'options'],
    num_rows: 120769
})

In [5]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [6]:
from tqdm import tqdm

{'sentences': ['With almost everything else to make them happy , they wanted one thing : they had no children .',
  'This vexed the king even more than the queen , who was very clever and learned , and who had hated dolls when she was a child .',
  'However , she , too in spite of all the books she read and all the pictures she painted , would have been glad enough to be the mother of a little prince .',
  'The king was anxious to consult the fairies , but the queen would not hear of such a thing .',
  'She did not believe in fairies : she said that they had never existed ; and that she maintained , though The History of the Royal Family was full of chapters about nothing else .',
  'Well , at long and at last they had a little boy , who was generally regarded as the finest baby that had ever been seen .',
  'Even her majesty herself remarked that , though she could never believe all the courtiers told her , yet he certainly was a fine child -- a very fine child .',
  'Now , the time d

In [54]:
example = train_set[1]
context, question, answer, options = example['sentences'], example['question'], example['answer'], example['options']
input_text = f"{context} {question} {' '.join(options)}"
tokenizer(input_text, add_special_tokens=True)

{'input_ids': [17816, 3152, 2048, 2279, 2073, 284, 787, 606, 3772, 837, 484, 2227, 530, 1517, 1058, 484, 550, 645, 1751, 764, 3256, 705, 1212, 41548, 276, 262, 5822, 772, 517, 621, 262, 16599, 837, 508, 373, 845, 14169, 290, 4499, 837, 290, 508, 550, 16563, 36062, 618, 673, 373, 257, 1200, 764, 3256, 705, 4864, 837, 673, 837, 1165, 287, 15275, 286, 477, 262, 3835, 673, 1100, 290, 477, 262, 5986, 673, 13055, 837, 561, 423, 587, 9675, 1576, 284, 307, 262, 2802, 286, 257, 1310, 19716, 764, 3256, 705, 464, 5822, 373, 18116, 284, 5725, 262, 3148, 444, 837, 475, 262, 16599, 561, 407, 3285, 286, 884, 257, 1517, 764, 3256, 705, 3347, 750, 407, 1975, 287, 3148, 444, 1058, 673, 531, 326, 484, 550, 1239, 11196, 2162, 290, 326, 673, 9456, 837, 996, 383, 7443, 286, 262, 8111, 7884, 373, 1336, 286, 15754, 546, 2147, 2073, 764, 3256, 705, 5779, 837, 379, 890, 290, 379, 938, 484, 550, 257, 1310, 2933, 837, 508, 373, 4143, 11987, 355, 262, 18822, 5156, 326, 550, 1683, 587, 1775, 764, 3256, 705, 6104, 6

task: write a preprocess function s.t. for an input: context, question, answer, options, output a format that we can put into data loader:
做成input_id, labels, 然后把input_id和labels都padding成一样长就行了应该。

In [8]:
train_set_lst = []
label_lst = []
attn_lst = []
padding = -100
max_length = 1024
for i in tqdm(range(len(train_set))):
    example = train_set[i]
    context, question, answer, options = example['sentences'], example['question'], example['answer'], example['options']
    input_text = f"{context} {question} {' '.join(options)}"
    input_ids,attention_mask = tokenizer(input_text, add_special_tokens=True)
    labels = [0]*len(input_ids)
    answer_id = tokenizer.encode(answer)[0]
    labels[-len(options) + options.index(answer)] = answer_id
    if len(input_ids)<=1024:
        padding_length = max_length-len(input_ids)
        input_ids+=[padding]*padding_length
        labels+=[padding]*padding_length
        attention_mask+=[padding]*padding_length
        train_set_lst.append(torch.tensor(input_ids))
        label_lst.append(torch.tensor(labels))
        attn_lst.append(attention_mask)


100%|██████████| 120769/120769 [04:44<00:00, 425.19it/s]


In [31]:
dataset = torch.utils.data.TensorDataset(*train_set_lst,*attn_lst,*label_lst)
train_dataloader = DataLoader(dataset,batch_size=8,shuffle=True)
print('finish')

finish


In [40]:
len(train_set_lst[1])

1024

In [42]:
for batch in train_dataloader:


AttributeError: 'list' object has no attribute 'size'

In [32]:

# Define model and optimizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
optimizer = AdamW(model.parameters(), lr=2e-5)

# Define training loop
def train(model, train_dataloader, optimizer, scheduler=None, num_epochs=1):
    model.train()
    for epoch in tqdm(range(num_epochs)):
        total_loss = 0
        #就这个下面一行我不知道要怎么把train_dataloader里面的东西读出来。。应该是要input_ids, attention_mask, 和labels,三个tensor
        for input_ids, labels in train_dataloader:

            input_ids, labels = input_ids.to(device), labels.to(device)
            outputs = model(input_ids, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        print(f"Epoch {epoch+1} Loss: {total_loss/len(train_dataloader)}")

# Load dataset and create dataloader


# Fine-tune model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
num_training_steps = len(train_dataloader) * 3
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)
train(model, train_dataloader, optimizer, scheduler=scheduler, num_epochs=100)

  0%|          | 0/100 [00:09<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [18]:
len(train_dataloader)

72